# Chapter 11: GRPO를 활용한 추론 모델 개발

## 1. 학습 목표

* GRPO의 원리와 기존 RLHF(PPO)와의 차이점을 이해한다.
* 추론 과정(Chain of Thought)을 유도하는 시스템 프롬프트를 설계한다.
* 정답 여부와 포맷 준수를 평가하는 보상 함수(Reward Function)를 구현한다.
* `GRPOTrainer`를 사용하여 수학 문제 해결 능력을 갖춘 모델을 학습한다.

## 2. GRPO (Group Relative Policy Optimization)란?

### 2.1 핵심 아이디어

기존의 강화학습(PPO)은 각 상태마다의 기대 보상을 예측하는 '가치 모델(Critic)'이 필요하여 메모리를 많이 차지하고 학습이 불안정했다.
**GRPO**는 동일한 질문에 대해 모델이 **N개의 답변을 생성**하게 한 뒤, 그 그룹 내에서 상대적으로 더 좋은 답변의 확률을 높이고 나쁜 답변의 확률을 낮추는 방식이다.

### 2.2 작동 프로세스

1. **생성**: 하나의 질문에 대해 답변 4~16개를 생성한다.
2. **평가**: 규칙 기반(Rule-based) 보상 함수로 각 답변을 채점한다 (예: 정답 맞춤=1, 틀림=0).
3. **업데이트**: 그룹 평균보다 점수가 높은 답변은 강화하고, 낮은 답변은 억제한다.

## 3. 추론 프롬프트 및 보상 함수 설계

모델이 "생각하는 과정"을 출력하도록 유도하기 위해 특수한 포맷을 정의한다.

### 3.1 시스템 프롬프트

In [1]:
# 추론을 위한 시스템 프롬프트
SYSTEM_PROMPT = """당신은 수학 문제를 풀 때 사고 과정을 명확히 보여주는 AI입니다.
1. 문제를 분석하고 해결하는 과정을 <THINK>와 </THINK> 태그 사이에 작성하세요.
2. 최종 정답은 <SOLUTION>과 </SOLUTION> 태그 사이에 작성하세요.
"""

# 예시 출력 포맷
example_output = """
<THINK>
문제: 48 + 27을 계산하라.
1. 48을 40 + 8로, 27을 20 + 7로 분리한다.
2. 십의 자리: 40 + 20 = 60
3. 일의 자리: 8 + 7 = 15
4. 합계: 60 + 15 = 75
</THINK>
<SOLUTION>
75
</SOLUTION>
"""
print("추론 프롬프트 포맷 설정 완료")

추론 프롬프트 포맷 설정 완료


### 3.2 보상 함수 구현

GRPO의 핵심은 **검증 가능한 보상**이다. 여기서는 두 가지 보상을 정의한다.

1. **Format Reward**: 지정된 태그(`<THINK>`, `<SOLUTION>`)를 지켰는가?
2. **Correctness Reward**: 최종 답이 정답과 일치하는가?

In [2]:
import re

def extract_solution(text):
    """
    모델 응답에서 <SOLUTION> 태그 안의 내용을 추출하는 함수다.
    """
    if "<SOLUTION>" in text and "</SOLUTION>" in text:
        answer = text.split("<SOLUTION>")[-1].split("</SOLUTION>")[0]
        return answer.strip()
    return None

def correctness_reward(prompts, completions, answer, **kwargs):
    """
    정답 여부를 판단하는 보상 함수다.
    """
    rewards = []
    for completion, gold_answer in zip(completions, answer):
        # 모델 응답에서 답 추출
        response_text = completion[0]['content'] if isinstance(completion, list) else completion
        extracted = extract_solution(response_text)

        # 정답 비교 (간단한 문자열 매칭)
        if extracted and extracted == str(gold_answer):
            rewards.append(1.0) # 정답
        else:
            rewards.append(0.0) # 오답

    return rewards

def format_reward(prompts, completions, **kwargs):
    """
    포맷 준수 여부를 판단하는 보상 함수다.
    """
    rewards = []
    pattern = r"<THINK>.*</THINK>.*<SOLUTION>.*</SOLUTION>"

    for completion in completions:
        response_text = completion[0]['content'] if isinstance(completion, list) else completion
        # 태그 구조와 줄바꿈 등을 유연하게 매칭 (DOTALL 옵션)
        if re.search(pattern, response_text, re.DOTALL):
            rewards.append(0.5) # 포맷 준수 보너스
        else:
            rewards.append(0.0)

    return rewards

print("보상 함수(정답, 포맷) 정의 완료")

보상 함수(정답, 포맷) 정의 완료


## 4. 데이터셋 준비 (GSM8K)

수학 추론 능력을 학습하기 위해 **GSM8K** 데이터셋을 사용한다.

In [3]:
from datasets import load_dataset

def prepare_gsm8k_dataset():
    """
    GSM8K 데이터셋을 로드하고 GRPO 학습용으로 변환하는 함수다.
    """
    # 데이터셋 로드 (실습용 500개)
    dataset = load_dataset('openai/gsm8k', 'main', split='train[:1000]')

    def process_data(example):
        # GSM8K 정답은 '#### 42' 형태이므로 숫자만 추출
        gold_answer = example['answer'].split('####')[-1].strip()

        return {
            # 프롬프트 구성 (시스템 메시지 포함)
            'prompt': [
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': example['question']}
            ],
            'answer': gold_answer
        }

    return dataset.map(process_data)

dataset = prepare_gsm8k_dataset()
print(f"데이터셋 준비 완료: {len(dataset)}개")
print(f"샘플: {dataset[0]['prompt']}")

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

데이터셋 준비 완료: 1000개
샘플: [{'content': '당신은 수학 문제를 풀 때 사고 과정을 명확히 보여주는 AI입니다.\n1. 문제를 분석하고 해결하는 과정을 <THINK>와 </THINK> 태그 사이에 작성하세요.\n2. 최종 정답은 <SOLUTION>과 </SOLUTION> 태그 사이에 작성하세요.\n', 'role': 'system'}, {'content': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'role': 'user'}]


## 5. 모델 로드 및 GRPO 학습 설정

`trl` 라이브러리의 `GRPOTrainer`를 사용한다. 베이스 모델은 `gpt-oss-20b` (실습용 `Qwen/Qwen2.5-14B`)를 사용하며, 메모리 절약을 위해 QLoRA를 적용한다.

In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import GRPOTrainer, GRPOConfig

# 1. 모델 로드 (QLoRA)
model_id = "openai/gpt-oss-20b"

# #model_id = "Qwen/Qwen3-14B"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True
# )

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    #quantization_config=bnb_config, # # Qwen3-14B의 경우 bnb_config와 함께 실행
    device_map="auto",
    attn_implementation="eager" # Qwen3-14B의 경우 "sdpa"
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# 2. LoRA 설정
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# 3. GRPO 학습 설정
training_args = GRPOConfig(
    output_dir="./GPT-OSS-20B-GRPO",
    learning_rate=5e-6,             # 매우 낮은 학습률 권장
    num_generations=4,              # 질문당 생성할 답변 수 (그룹 크기)
    max_completion_length=512,      # 생성 답변 최대 길이
    per_device_train_batch_size=1,  # 생성 메모리 부담이 크므로 배치는 작게
    gradient_accumulation_steps=4,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_steps=50,
    report_to="none"
)

# 4. Trainer 생성
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    reward_funcs=[correctness_reward, format_reward], # 정의한 보상 함수 리스트
    args=training_args,
    train_dataset=dataset,
)

print("GRPO 학습 준비 완료")

MXFP4 quantization requires Triton and kernels installed: CUDA requires Triton >= 3.4.0, XPU requires Triton >= 3.5.0, we will default to dequantizing the model to bf16


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

GRPO 학습 준비 완료


## 6. 학습 실행

In [ ]:
print("GRPO 학습 시작...")
# GRPO는 생성 과정이 포함되어 있어 SFT보다 시간이 더 소요된다.
trainer.train()
print("학습 완료!")

# 모델 저장
trainer.save_model("./GPT-OSS-20B-GRPO-Reasoning")

## 7. 요약

이 챕터에서는 최신 추론 모델 학습 기법인 **GRPO**를 실습했다.

1. **그룹 상대 평가**: 복잡한 가치 모델 없이, 생성된 답변들끼리의 비교만으로 학습이 가능하다.
2. **보상 함수 설계**: 정답(Correctness)과 형식(Format)이라는 명확한 기준으로 모델을 유도했다.
3. **추론 능력 강화**: `<THINK>` 태그를 통해 모델이 사고 과정을 거치도록 훈련시켰다.

이로써 기초적인 정렬부터 고급 추론 학습까지 마쳤다. 다음 챕터부터는 학습된 모델의 성능을 더 끌어올리거나 최적화하는 고급 기법들을 다룬다.

다음 챕터는 **Chapter 12: 고급 LoRA 기법**으로, DoRA, AdaLoRA 등 성능과 효율을 더 높이는 PEFT의 변형 기법들을 다룬다.